# Tools for SUITE Risk-Limiting Election Audits



In [1]:
from __future__ import print_function

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display

from collections import OrderedDict
from itertools import product
import math

import numpy as np
from ballot_comparison import ballot_comparison_pvalue
from fishers_combination import fisher_combined_pvalue, maximize_fisher_combined_pvalue, \
     bound_fisher_fun, calculate_lambda_range    
from sprt import ballot_polling_sprt

from cryptorandom.cryptorandom import SHA256
from cryptorandom.sample import sample_by_index

In [2]:
# global audit parameters

seed = 12345678901234567890  # use, e.g., 20 rolls of a 10-sided die
risk_limit = 0.05    # risk limit

gamma=1.03905         # gamma from Lindeman and Stark (2012)

lambda_step = 0.05    # stepsize for the discrete bounds on Fisher's combining function

# assumptions for finding initial sample sizes

o1_rate = 0.002       # expect 2 1-vote overstatements per 1000 ballots in the CVR stratum
o2_rate = 0           # expect 0 2-vote overstatements
u1_rate = 0           # expect 0 1-vote understatements
u2_rate = 0           # expect 0 2-vote understatements

sample_increment = 10 # step size (in ballots) for estimating initial sample sizes.
                      # The sample is increased in increments of sample_increment until
                      # the maximum combined p-value is expected to be below the risk limit,
                      # on the assumption that the results are correct in the no-CVR stratum
                      # and subject to assumptions about the expected rate of overstatement
                      # and understatement errors in the CVR stratum

stratum_sizes = [100000, 5000]  # total ballots in the two strata, CVR, no-CVR

n_ratio = stratum_sizes[0]/np.sum(stratum_sizes) 
                     # allocate sample in proportion to ballots cast in each stratum

# contest-specific parameters
num_winners = 2       # maximum number of winners, per social choice function

# Initial sample size

Reported Votes

In [3]:
# input number of winners
# input names as well as reported votes in each stratum

# candidates are a dict with name, [votes_in_stratum_1, votes_in_stratum_2]
candidates = { "candidate 3": [30000, 500],
               "candidate 2": [50000, 1000],
               "candidate 1": [10000, 500],
               "candidate 4": [500, 10]}

cvr_votes = poll_votes = 0

for name, votes in candidates.items():
    cvr_votes += votes[0]
    poll_votes += votes[1]
    votes.append(votes[0]+votes[1])

In [4]:
assert cvr_votes <= stratum_sizes[0]
assert poll_votes <= stratum_sizes[1]
assert (n_ratio >= 0) and (n_ratio <= 1)

In [5]:
# find reported winners, losers, and pairwise margins

candidates = OrderedDict(sorted(candidates.items(), key=(lambda t: t[1][2]), reverse=True))
                        
winners = list(candidates.keys())[0:num_winners]
losers = list(candidates.keys())[num_winners:]

votes = np.zeros(3)

margins = {}  # will hold the (winner, loser) pairwise margins indexed by pairs

for x in product(winners, losers):
    margins[x] = candidates[x[0]][2] - candidates[x[1]][2]

margins = OrderedDict(sorted(margins.items(), key=(lambda t: t[1]), reverse=True))

min_margin = np.amin(list(margins.values()))

print('\nTotal reported votes:\n\t\t\tCVR\tno-CVR\ttotal')
for k, v in candidates.items():
    print('\t', k, ':', v[0], '\t', v[1], '\t', v[2])
print('\n\t total votes:\t', cvr_votes, '\t', poll_votes, '\t', cvr_votes + poll_votes)
print('\n\t non-votes:\t',\
      stratum_sizes[0] - cvr_votes,'\t',\
      stratum_sizes[1] - poll_votes,'\t',\
      stratum_sizes[0] + stratum_sizes[1] - cvr_votes - poll_votes\
     )



print('\nwinners:')
for w in winners:
    print('\t',w)

print('\nlosers:')
for ell in losers:
    print('\t',ell)

print('\n\nmargins:')
for k, v in margins.items():
    dum = k[0] + ' beat ' + k[1] + ' by'
    print('\t', dum , v, 'votes')

print('\nsmallest margin:', min_margin, '\ndiluted margin:', min_margin/np.sum(stratum_sizes))


Total reported votes:
			CVR	no-CVR	total
	 candidate 2 : 50000 	 1000 	 51000
	 candidate 3 : 30000 	 500 	 30500
	 candidate 1 : 10000 	 500 	 10500
	 candidate 4 : 500 	 10 	 510

	 total votes:	 90500 	 2010 	 92510

	 non-votes:	 9500 	 2990 	 12490

winners:
	 candidate 2
	 candidate 3

losers:
	 candidate 1
	 candidate 4


margins:
	 candidate 2 beat candidate 4 by 50490 votes
	 candidate 2 beat candidate 1 by 40500 votes
	 candidate 3 beat candidate 4 by 29990 votes
	 candidate 3 beat candidate 1 by 20000 votes

smallest margin: 20000 
diluted margin: 0.19047619047619047


Expected sample sizes

In [6]:
def estimate_n(Nw1, Nw2, Nl1, Nl2, N1, N2, o1_rate=0, o2_rate=0, u1_rate=0, u2_rate=0,
               n_ratio=None,
               risk_limit=0.05,\
               gamma=1.03905,\
               sample_increment=10, stepsize=0.05):
    '''
    Estimate the initial sample sizes for the audit.
    
    TO DO: Document this!
    
    '''
    n_ratio = n_ratio if n_ratio else N1/(N1+N2)
    n = 0
    reported_margin = (Nw1+Nw2)-(Nl1+Nl2)
    expected_pvalue = 1
    while (expected_pvalue > risk_limit) or (expected_pvalue is np.nan):
    # TO DO: Replace linear search with bisection search. First, increase n by doubling
    #  until it's large enough; then use bisection between the last size that failed 
    # and the first size that succeeded to find the (integer) solution. Be sure to limit
    # trial values of n to integers, using ceil or floor
        n = n + sample_increment
        n1 = math.ceil(n_ratio * n)
        n2 = n - n1
        o1 = math.ceil(o1_rate*n1)
        o2 = math.ceil(o2_rate*n1)
        u1 = math.floor(u1_rate*n1)
        u2 = math.floor(u2_rate*n1)

        cvr_pvalue = lambda alloc: ballot_comparison_pvalue(n=n1, gamma=1.03905, o1=o1, 
                                                    u1=u1, o2=o2, u2=u2, 
                                                    reported_margin=reported_margin, N=N1, 
                                                    null_lambda=alloc)

        nocvr_pvalue = lambda alloc: ballot_polling_sprt(sample= np.array([0]*int(n2*Nl2/N2)+\
                                             [1]*int(n2*Nw2/N2)+\
                                             [np.nan]*int(n2*(N2-Nl2-Nw2)/N2)), \
                            popsize=N2, \
                            alpha=risk_limit,\
                            Vw=Nw2, Vl=Nl2, null_margin=(Nw2-Nl2) - alloc*reported_margin)['pvalue']
        # Crude maximizer for now 
        # TO DO: this isn't rigorous yet--needs to be fixed
        res = bound_fisher_fun(Nw1, Nl1, N1, Nw2, Nl2, N2,
                       pvalue_funs=(cvr_pvalue, nocvr_pvalue),\
                       stepsize=stepsize, feasible_lambda_range=None)
        expected_pvalue = np.amax(res['upper_bounds'])
        if (n % 10000)==0:
            print('...trying...', n, expected_pvalue)    
    return (n1, n2)

In [7]:
# Find largest expected sample size across (winner, loser) pairs

sample_sizes = {}

for k in product(winners, losers):
    sample_sizes[k] = estimate_n(candidates[k[0]][0],\
                                 candidates[k[0]][1],\
                                 candidates[k[1]][0],\
                                 candidates[k[1]][1],\
                                 stratum_sizes[0],\
                                 stratum_sizes[1],\
                                 o1_rate = o1_rate,\
                                 o2_rate = o2_rate,\
                                 u1_rate = u1_rate,\
                                 u2_rate = u2_rate,\
                                 n_ratio = n_ratio,\
                                 risk_limit = risk_limit,\
                                 gamma = gamma,\
                                 sample_increment = sample_increment,\
                                 stepsize = lambda_step)

In [8]:
sample_size = np.amax([v[0]+v[1] for v in sample_sizes.values()])

print(sample_sizes, '\nexpected minimum sample size:', sample_size)

{('candidate 2', 'candidate 1'): (39, 1), ('candidate 2', 'candidate 4'): (29, 1), ('candidate 3', 'candidate 1'): (67, 3), ('candidate 3', 'candidate 4'): (48, 2)} 
expected minimum sample size: 70


# Random sampling

In [9]:
prng = SHA256(seed)   # initialize the PRNG

In [10]:
n1 = math.ceil(sample_size*n_ratio)    # stratum 1 initial sample size

sample1 = sample_by_index(stratum_sizes[0], n1, prng)
sample2 = sample_by_index(stratum_sizes[1], sample_size-n1, prng)

Stratum 1 sample

In [11]:
print("CVR stratum sample:\n", sample1)

CVR stratum sample:
 {29184, 20483, 93197, 11301, 28716, 57905, 73787, 50747, 27199, 62020, 81480, 34387, 48726, 91743, 94318, 56943, 45174, 84601, 43658, 84619, 60569, 95386, 72874, 24761, 9425, 85218, 3313, 79101, 82699, 49937, 90405, 27949, 39218, 92466, 32579, 22855, 25425, 95578, 16228, 6505, 29561, 85882, 19322, 99715, 40326, 95623, 15242, 79258, 55196, 64414, 4000, 16292, 41396, 27574, 61370, 43463, 57810, 21458, 41938, 94685, 50669, 26094, 94703, 67570, 48627, 54775, 52732}


In [12]:
print("CVR stratum sample, sorted:\n", np.sort(sample1))

AxisError: axis -1 is out of bounds for array of dimension 0

In [ ]:
## TO DO: either sample with replacment, or replace the math
print("CVR stratum sample, sorted, duplicates removed:\n", np.unique(np.sort(sample1)))

In [ ]:
m = np.zeros_like(sample1, dtype=bool)
m[np.unique(sample1, return_index=True)[1]] = True
print("Stratum 1 repeated ballots:\n", sample1[~m])

Stratum 2 sample

In [ ]:
print("No-CVR stratum sample:\n", sample2)

In [ ]:
print("No-CVR stratum sample, sorted:\n", np.sort(sample2))

## TO DO: double check that ballot polling SPRT test is without replacement

In [ ]:
m2 = np.zeros_like(sample2, dtype=bool)
m2[np.unique(sample2, return_index=True)[1]] = True
print("Stratum 2 repeated ballots:\n", sample2[~m2])

# Find ballots using ballot manifest

Ballot manifest: Each line must have a batch label, a comma, and one of the following:
  1. the number of ballots in the batch 
  1. a range specified with a colon (e.g., 131:302), or 
  1. a list of ballot identifiers within parentheses, separated by spaces (e.g., (996 998 1000)).
  
Each line should have exactly one comma.

In [ ]:
# I'm imagining this is is a list for now
ballot_manifest_cvr = ['1, 100', '2, 101:200', '3, (205 210)']
ballot_manifest_poll = ['1, 100', '2, 101:200', '3, (205 210)']

In [ ]:
# step 1: expand the ballot manifest into a dict. keys are batches, values are ballot numbers.

def parse_manifest(manifest):
    '''
    Parses a ballot manifest to put the batches in a canonical ordering and be able
    to look up specific ballots.
    
    Input: 
      a ballot manifest in the syntax described above
    
    Returns:
      an ordered dict containing
         batch ID (key)
         ballot identifiers within the batch, either from sequential enumeration or from
             the given labels
    '''
    ballots = 0
    ballot_manifest_dict = OrderedDict()
    for i in manifest:
        # assert that the entry is a string with a comma in it
        # pull out batch label
        (batch, val) = i.split(",")
        batch = batch.strip()
        val = val.strip()    
        if (batch in ballot_manifest_dict.keys()):
             raise ValueError('batch is listed more than once')
        else:
            ballot_manifest_dict[batch] = []
    
        # parse what comes after the batch label
        if '(' in val:     # list of identifiers
            val = val[1:-1] # strip out the parentheses  TO DO: use regex to remove )(
            ballot_manifest_dict[batch] += [int(num) for num in val.split()]
            
        elif ':' in val:   # range of identifiers
            limits = val.split(':')
            ballot_manifest_dict[batch] += list(range(int(limits[0]), int(limits[1])+1))  
            
        else:  # this should be an integer number of ballots
            try:
                ballot_manifest_dict[batch] += list(range(1, int(val)+1))
            except:
                print('malformed row in ballot manifest:\n\t', i)
    return(ballot_manifest_dict)

In [ ]:
cvr_manifest_parsed = parse_manifest(ballot_manifest_cvr)
poll_manifest_parsed = parse_manifest(ballot_manifest_poll)

listed_cvr = np.sum([len(v) for v in cvr_manifest_parsed.values()])
listed_poll = np.sum([len(v) for v in poll_manifest_parsed.values()])

In [ ]:
# test that manifest matches reported ballot totals

assert listed_cvr == stratum_sizes[0]
assert listed_poll == stratum_sizes[1]

In [ ]:
# step 2: look up sample values

def find_ballot(ballot_num):
    for batch, ballots in ballot_manifest_dict.items():
        if ballot_num in ballots:
            position = ballots.index(ballot_num)
            return batch, position
    print("Ballot %i not found" % ballot_num)
    return None

print("sorted number, ballot, batch_label, which_ballot_in_batch")
i = 0
for s in sample1:
    i += 1
    batch_label, which_ballot = find_ballot(s)
    print(i, s, batch_label, which_ballot) # This uses 0-indexing still. Should we change it be 1,...,n?

# Should more ballots be audited?

Sample statistics for the CVR stratum (stratum 1)

In [ ]:
# Simple version, fill in the blanks

o1 = 
u1 = 
o2 = 
u2 = 

In [ ]:
# Tricky version, input using sliders

def stratum1_inputs(o1, u1, o2, u2):
    return (o1, u1, o2, u2)

stratum1_stats = interactive(stratum1_inputs, 
                             o1 = widgets.IntSlider(min=0,max=n1,value=0),
                             u1 = widgets.IntSlider(min=0,max=n1,value=0),
                             o2 = widgets.IntSlider(min=0,max=n1,value=0),
                             u2 = widgets.IntSlider(min=0,max=n1,value=0))
display(stratum1_stats)

In [ ]:
(o1, u1, o2, u2) = [stratum1_stats.children[i].value for i in range(4)]

Sample statistics for the no-CVR stratum (stratum 2)

In [ ]:
# Simple version, fill in the blanks


n2l =
n2w = 

In [ ]:
# Tricky version, input using sliders

def stratum2_inputs(ballots_for_loser, ballots_for_winner):
    return (ballots_for_loser, ballots_for_winner)

stratum2_stats = interactive(stratum2_inputs, 
                             ballots_for_loser = widgets.IntSlider(min=0,max=n2,value=0),
                             ballots_for_winner = widgets.IntSlider(min=0,max=n2,value=0))
display(stratum2_stats)

In [ ]:
(n2l, n2w) = [stratum2_stats.children[i].value for i in range(2)]

In [ ]:
cvr_pvalue = lambda alloc: ballot_comparison_pvalue(n=n1, gamma=1.03905, o1=o1, 
                                                    u1=u1, o2=o2, u2=u2, 
                                                    reported_margin=reported_margin, N=N1, 
                                                    null_lambda=alloc)
nocvr_pvalue = lambda alloc: ballot_polling_sprt(sample= np.array([0]*n2l+[1]*n2w+[np.nan]*(n2-n2w-n2l)), \
                            popsize=N2, \
                            alpha=0.05,  # set this param but we don't need to use it
                            Vw=Nw2, Vl=Nl2, null_margin=(Nw2-Nl2) - alloc*reported_margin)['pvalue']
# Crude maximizer for now
res = bound_fisher_fun(Nw1, Nl1, N1, Nw2, Nl2, N2,
                       pvalue_funs=(cvr_pvalue, nocvr_pvalue), stepsize=lambda_step,\
                       feasible_lambda_range=None
expected_pvalue = np.max(res['upper_bounds'])
if expected_pvalue <= alpha:
    print("Stop the audit")
else:
    print("Escalate the audit")